Họ và tên: Phạm Đức Thể

MSSV: 19522253

Lớp: DS300.N11

LAB04 - 31/10/2022

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1S9wvDrnX-ePP_yMJlsC-dBLgXkCsS47y?usp=sharing)

# **LAB 04 - CẢI THIỆN CONTENT-BASED BẰNG WORD-EMBEDDING ĐÁNH GIÁ KHUYẾN NGHỊ**

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
import time
import math

from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import norm
from numpy import dot
from numpy import sqrt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

pd.options.display.max_colwidth = 200
sns.set()

In [2]:
PATH = '/content/drive/MyDrive/TÀI LIỆU HỌC TẬP ĐẠI HỌC 2019-2023/NĂM 4 2022-2023/HỌC KỲ 1 2022 - 2023/DS300.N11 - HỆ KHUYẾN NGHỊ/THỰC HÀNH/LAB/LAB04 - 31 10 2022/dataset'

## Cài đặt lại bài tập thực hành số 2 (CF) trên bộ dữ liệu MovieLens.

In [3]:
# load dữ liệu từ file ratings
ratings = pd.read_csv(PATH+'/ratings_s.csv')
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [4]:
X_train, X_test = train_test_split(ratings, test_size=0.002, random_state=42)
print('Size Train = ', len(X_train))
print('Size Test = ', len(X_test))

Size Train =  100634
Size Test =  202


In [5]:
X_train

,userId,movieId,rating,timestamp
56860,379,349,4.0,847397292
30433,212,96079,4.0,1523216972
23644,160,4979,4.5,1058456459
18829,121,597,3.0,847656027
39763,274,3285,2.0,1171786187
...,...,...,...,...
6265,42,4005,4.0,996259059
54886,364,141,4.0,869443367
76820,480,6867,4.0,1179163171
860,6,981,3.0,845556567


In [6]:
X_test

,userId,movieId,rating,timestamp
67037,432,77866,4.5,1335139641
42175,288,474,3.0,978465565
93850,599,4351,3.0,1498524542
6187,42,2987,4.0,996262677
12229,75,1610,4.0,1158989841
...,...,...,...,...
56055,370,4014,3.0,1159164363
63609,414,4103,5.0,991144129
73428,474,1223,4.5,1053372016
77083,481,1653,3.0,884773369


In [7]:
# load dữ liệu từ file movies
movies = pd.read_csv(PATH+'/movies.csv')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [8]:
# merge bảng ratings, và movies theo cột "movieID"
ratings_movies = pd.merge(X_train, movies, on='movieId')
ratings_movies

,userId,movieId,rating,timestamp,title,genres
0,379,349,4.0,847397292,Clear and Present Danger (1994),Action|Crime|Drama|Thriller
1,174,349,4.0,848486943,Clear and Present Danger (1994),Action|Crime|Drama|Thriller
2,181,349,4.0,845469325,Clear and Present Danger (1994),Action|Crime|Drama|Thriller
3,43,349,3.0,848993694,Clear and Present Danger (1994),Action|Crime|Drama|Thriller
4,374,349,3.0,849089103,Clear and Present Danger (1994),Action|Crime|Drama|Thriller
...,...,...,...,...,...,...
100043,462,58191,4.0,1265662119,Taxi to the Dark Side (2007),Documentary
100044,265,3807,3.0,965314642,Sinbad and the Eye of the Tiger (1977),Adventure|Fantasy
100045,89,136353,5.0,1520409074,Scooby-Doo! and the Loch Ness Monster (2004),Animation|Children|Comedy
100046,596,113280,4.0,1535833142,White Frog (2012),Drama


In [9]:
# Chuyển bảng ratings sang dataframe với các cột là "userId" và các hàng là "movieId" (*)
ratings_df = X_train.pivot(index='movieId', columns='userId')['rating'].fillna(0)
ratings_df

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Xây dựng thuật toán lọc cộng tác dựa trên User - Hệ số tương quan Pearson

In [10]:
# Lấy ra rating của 1 user cho 1 movie
def get_rating(userid, movieid):
  return (X_train.loc[(X_train.userId==userid) & (X_train.movieId==movieid), 'rating'].iloc[0])

In [11]:
get_rating(7,1)

4.5

In [12]:
def pearson_correlation_score(user1, user2):
  both_watch_count = [] # Danh sách các bộ phim được xem chung bởi user1 và user2
  list_movie_user1 = X_train.loc[X_train.userId==user1, 'movieId'].to_list()
  list_movie_user2 = X_train.loc[X_train.userId==user2, 'movieId'].to_list()

  for movieid in list_movie_user1:
    if movieid in list_movie_user2:
      both_watch_count.append(movieid)
  
  if len(both_watch_count)==0:
    return 0
  
  sum_rating_1 = sum([get_rating(user1, i) for i in both_watch_count])
  avg_rating_1 = sum_rating_1/len(both_watch_count) # rating trung bình user1

  sum_rating_2 = sum([get_rating(user2, i) for i in both_watch_count])
  avg_rating_2 = sum_rating_2/len(both_watch_count) # rating trung bình user2

  TuSo = sum([(get_rating(user1, i) - avg_rating_1)*(get_rating(user2, i) - avg_rating_2) for i in both_watch_count])
  MauSo = sqrt(sum((get_rating(user1, i) - avg_rating_1)**2 for i in both_watch_count))*sqrt(sum((get_rating(user2, i) - avg_rating_2)**2 for i in both_watch_count))

  if MauSo==0:
    return 0
  
  return TuSo/MauSo

In [13]:
pearson_correlation_score(50,200)

0.39605699951716145

### Xây dựng thuật toán lọc cộng tác dựa trên Item - Độ đo tương tự cosine

In [14]:
def distance_similarity_score(user1, user2):
  both_watch_count = 0
  list_movie_user1 = X_train.loc[X_train.userId==user1, 'movieId'].tolist()
  list_movie_user2 = X_train.loc[X_train.userId==user2, 'movieId'].tolist()

  for movieid in list_movie_user1:
    if movieid in list_movie_user2:
      both_watch_count += 1
  
  if both_watch_count == 0:
    return 0

  rating1 = []
  rating2 = []

  for movieid in list_movie_user1:
    if movieid in list_movie_user2:
      rating1.append(get_rating(user1, movieid))
      rating2.append(get_rating(user2, movieid))
  
  # print(rating1)
  # print(rating2)

  return dot(rating1, rating2)/(norm(rating1)*norm(rating2))

In [15]:
print('Distance based similarity between user ids X & u1: {}'.format(distance_similarity_score(1,2)))

Distance based similarity between user ids X & u1: 0.9999999999999998


In [16]:
print('Distance based similarity between user ids X & u2: {}'.format(distance_similarity_score(1,3)))

Distance based similarity between user ids X & u2: 0.8342365272454381


In [17]:
print('Distance based similarity between user ids X & u3: {}'.format(distance_similarity_score(1,4)))

Distance based similarity between user ids X & u3: 0.9328095891279707


### Xây dựng thuật toán lọc cộng tác dựa trên User với độ tương tự cosine
 Xuất ra top 10 bộ phim (chưa được người dùng xem) có dự đoán rating cao nhất cho UserID =20

In [18]:
def most_similar_user(user1, similarity_name, number_of_user=100):
  userid = X_train.userId.unique().tolist()
  # print(len(userid))

  if similarity_name == 'pearson':
    similarity_score =  [(pearson_correlation_score(user1, user_i), user_i) for user_i in userid if user_i != user1]

  if similarity_name == 'cosine':
    similarity_score =  [(distance_similarity_score(user1, user_i), user_i) for user_i in userid if user_i != user1]

  similarity_score.sort() # Tăng dần
  similarity_score.reverse() # Giảm dần

  return similarity_score[:number_of_user] # Có thể thay đổi số lượng lân cận

In [19]:
list_user_popular = most_similar_user(20, "pearson", 10)
list_user_popular

[(1.0000000000000002, 472),
 (1.0000000000000002, 378),
 (1.0000000000000002, 209),
 (1.0000000000000002, 127),
 (1.0000000000000002, 54),
 (1.0, 511),
 (1.0, 498),
 (1.0, 442),
 (1.0, 364),
 (1.0, 337)]

In [20]:
# Danh sách các bộ phim đã rating bởi người dùng
def get_movieids(userid):
  return (X_train.loc[(X_train.userId==userid), 'movieId'].tolist())

In [21]:
print(get_movieids(20))

[5171, 783, 5618, 1566, 709, 4821, 661, 2081, 3159, 1380, 1030, 1097, 3793, 4161, 4154, 3438, 1547, 2087, 4447, 3751, 3948, 4901, 3565, 3967, 5991, 2033, 3483, 551, 5504, 3972, 2761, 4993, 2687, 1688, 3398, 5364, 4865, 837, 3999, 4720, 1333, 2018, 2116, 6365, 3300, 5679, 34, 1919, 4846, 2135, 899, 1012, 914, 4366, 6157, 4029, 1282, 4848, 6297, 4370, 4022, 4446, 1029, 4344, 5254, 3512, 5015, 5218, 4979, 3510, 3852, 2701, 5110, 2941, 6143, 1367, 4369, 5299, 1021, 6333, 2096, 2300, 1722, 588, 1018, 1025, 4025, 801, 4027, 1013, 3034, 4299, 4449, 3897, 1088, 2770, 2746, 5463, 3996, 5103, 596, 48, 4886, 3624, 2092, 4896, 1489, 4014, 2657, 3986, 3114, 1028, 4995, 4734, 4638, 1907, 13, 3615, 107, 5313, 2394, 1947, 3536, 2140, 239, 4019, 4643, 4378, 1148, 2709, 2005, 1035, 1022, 4308, 594, 2857, 2085, 4776, 5785, 2430, 4039, 6156, 4718, 2080, 3549, 531, 2565, 1848, 362, 4367, 2054, 5502, 2006, 5013, 3623, 3753, 3994, 4270, 3755, 1032, 2294, 2078, 5419, 919, 4519, 4816, 4641, 5989, 4226, 3990, 2

In [22]:
# Lấy ra tên của một bộ phim
def get_movie_title(movieid):
  return movies.loc[(movies.movieId==movieid), 'title'].iloc[0]

In [23]:
get_movie_title(500)

'Mrs. Doubtfire (1993)'

In [24]:
# Lấy ra danh sách khuyến nghị từ top populars
def get_recommendation(userid, similarity_name='cosine'):
  total = {}
  sum_similarity = {}
  list_user_popular = most_similar_user(userid, similarity_name, number_of_user=100)

  # Iterating over subset of user ids.
  for similarityName, user in list_user_popular:
    score = similarityName
    movieids = get_movieids(user) 
    for movieid in movieids: # -> Danh sách các id movie đã xem bởi user khác và khởi tạo giá trị = 0
      if movieid not in get_movieids(userid):
        total[movieid] = 0
        sum_similarity[movieid] = 0

    for movieid in movieids: #-> Danh sách các id movie đã xem bởi user khác
      if movieid not in get_movieids(userid):
        total[movieid] += get_rating(user, movieid)*score
        sum_similarity[movieid] += score

  # Normalizing ratings
  recommendations = [(movieid, tot/sum_similarity[movieid]) for movieid, tot in total.items()]
  recommendations.sort()
  recommendations.reverse()

  # recommendations = [(movieid, score) for score, movieid in ranking]
  return recommendations

In [25]:
ls = get_recommendation(1)
ls

[(193587, 3.5),
 (193585, 3.5),
 (193583, 3.5),
 (193581, 4.0),
 (193579, 3.5),
 (193573, 4.0),
 (193571, 4.0),
 (193567, 3.0),
 (193565, 3.5),
 (191005, 4.5),
 (190221, 1.0),
 (190219, 1.0),
 (190215, 1.5),
 (190213, 1.0),
 (190209, 4.0),
 (190207, 1.5),
 (189333, 3.5),
 (189111, 3.0),
 (189043, 2.5),
 (188301, 4.0),
 (187595, 4.0),
 (187593, 1.0),
 (187541, 1.0),
 (185135, 4.999999999999999),
 (185031, 4.5),
 (185029, 1.0),
 (184997, 4.0),
 (184641, 0.5),
 (184471, 3.5000000000000004),
 (183911, 1.5),
 (183635, 3.5),
 (183611, 4.0),
 (182793, 3.5),
 (182715, 4.0),
 (181719, 3.5),
 (181413, 4.5),
 (180987, 1.5),
 (180985, 3.0),
 (180497, 4.5),
 (180297, 4.5),
 (180095, 5.0),
 (179819, 3.5000000000000004),
 (179817, 4.5),
 (179401, 3.5000000000000004),
 (178827, 5.0),
 (178615, 3.5),
 (178111, 2.0),
 (177939, 4.5),
 (177765, 2.5),
 (177763, 3.5),
 (177593, 5.0),
 (177285, 3.0),
 (176601, 4.999999999999999),
 (176415, 3.5000000000000004),
 (176413, 4.5),
 (176371, 1.0),
 (176101, 2.5),


In [26]:
len(ls)

1976

In [27]:
# hàm dữ đoán trên tập test 
def get_recommendation_4test(test):
  X_test_pred = []
  X_test_userid = X_test['userId'].tolist()   #lấy ra danh sách các userID trong tập test 
  X_test_movieID = X_test['movieId'].tolist()

  print("------Dự đoán cho tập test với {} điểm dữ liệu------".format(len(X_test_userid)))
  
  for i in range(len(X_test_userid)):        #lấy ra từng user
    list_R = get_recommendation(X_test_userid[i])    #lất ra danh sách khuyến nghị cho mỗi USER 
    check = 0
    for j in list_R:
      if(X_test_movieID[i] == j[0]):     #j[1] là lấy vị trí số 2 của 1 tuple 
        X_test_pred.append(j[1])
        check = 1
    if(check == 0):
      X_test_pred.append(0)
    print('----- Đang dự đoán cho dòng thứ {}'.format(i), 'là {}'.format(X_test_pred[i])) 
  return X_test_pred

In [28]:
start = time.time()
X_test_pred_rating = get_recommendation_4test(X_test)
end = time.time()
print(f'Tổng thời gian dự đoán: {(end - start)/60} minutes')

------Dự đoán cho tập test với 202 điểm dữ liệu------
----- Đang dự đoán cho dòng thứ 0 là 0
----- Đang dự đoán cho dòng thứ 1 là 4.0
----- Đang dự đoán cho dòng thứ 2 là 0
----- Đang dự đoán cho dòng thứ 3 là 4.0
----- Đang dự đoán cho dòng thứ 4 là 4.5
----- Đang dự đoán cho dòng thứ 5 là 0
----- Đang dự đoán cho dòng thứ 6 là 3.5
----- Đang dự đoán cho dòng thứ 7 là 3.0
----- Đang dự đoán cho dòng thứ 8 là 0
----- Đang dự đoán cho dòng thứ 9 là 0
----- Đang dự đoán cho dòng thứ 10 là 4.0
----- Đang dự đoán cho dòng thứ 11 là 4.0
----- Đang dự đoán cho dòng thứ 12 là 4.5
----- Đang dự đoán cho dòng thứ 13 là 5.0
----- Đang dự đoán cho dòng thứ 14 là 0
----- Đang dự đoán cho dòng thứ 15 là 4.0
----- Đang dự đoán cho dòng thứ 16 là 0
----- Đang dự đoán cho dòng thứ 17 là 0
----- Đang dự đoán cho dòng thứ 18 là 4.0
----- Đang dự đoán cho dòng thứ 19 là 4.0
----- Đang dự đoán cho dòng thứ 20 là 4.0
----- Đang dự đoán cho dòng thứ 21 là 0
----- Đang dự đoán cho dòng thứ 22 là 4.0
----- Đa

In [29]:
len(X_test_pred_rating)

202

In [31]:
X_test_pred_rating

[0,
 4.0,
 0,
 4.0,
 4.5,
 0,
 3.5,
 3.0,
 0,
 0,
 4.0,
 4.0,
 4.5,
 5.0,
 0,
 4.0,
 0,
 0,
 4.0,
 4.0,
 4.0,
 0,
 4.0,
 4.5,
 4.0,
 0,
 2.9999999999999996,
 0,
 3.5,
 0,
 0,
 0,
 4.0,
 4.5,
 0,
 2.0,
 0,
 5.0,
 4.0,
 0,
 4.0,
 4.0,
 3.0000000000000004,
 0,
 3.5,
 0,
 0,
 5.0,
 4.5,
 4.0,
 2.9999999999999996,
 5.0,
 4.5,
 0,
 5.0,
 5.0,
 0,
 4.0,
 0,
 5.0,
 4.5,
 3.0,
 0,
 4.0,
 0,
 0,
 0,
 3.5,
 0,
 4.0,
 0,
 3.5,
 0,
 5.0,
 0,
 0,
 3.0,
 0,
 0,
 4.5,
 0,
 0,
 3.5,
 3.5,
 4.5,
 4.5,
 2.0,
 4.0,
 0,
 2.5000000000000004,
 0,
 0,
 4.5,
 0,
 3.0000000000000004,
 4.0,
 4.0,
 5.0,
 4.0,
 0.5,
 3.5000000000000004,
 3.0,
 0,
 0,
 0,
 4.0,
 5.0,
 0,
 4.5,
 5.0,
 4.0,
 4.5,
 0,
 4.0,
 2.0,
 5.0,
 3.0,
 3.5,
 0,
 4.0,
 0,
 4.0,
 5.0,
 4.0,
 4.0,
 4.5,
 0,
 2.4999999999999996,
 4.0,
 0,
 4.5,
 0,
 5.0,
 0,
 3.5,
 0,
 0,
 0,
 5.0,
 0,
 0,
 0,
 0,
 3.0,
 0,
 0,
 0,
 0,
 0,
 3.0,
 0,
 3.5,
 4.0,
 0,
 4.5,
 3.5,
 0,
 5.0,
 1.0,
 0,
 4.5,
 5.0,
 4.0,
 3.0,
 0,
 3.0000000000000004,
 0,
 4.0,
 4.0,
 3.0

### Đánh giá theo các độ đo: MSE, RMSE, MAE, NMAE.

#### MSE

In [38]:
mse = mean_squared_error(X_test.rating , X_test_pred_rating)
print('MSE =',mse)

MSE = 4.930693069306931


#### RMSE

In [39]:
rmse = math.sqrt(mse)
print('RMSE =',rmse)

RMSE = 2.2205163969912336


#### MAE

In [40]:
mae = mean_absolute_error(X_test.rating , X_test_pred_rating)
print('MAE: ',mae)

MAE:  1.7425742574257426


#### NMAE

In [43]:
X_test['rating'].tolist().sort(reverse=True)
max = X_test['rating'].tolist()[0]
min = X_test['rating'].tolist()[len(X_test['rating'].tolist()) -1]

nmae = mae/(max-min)
print('----- NMAE: ',nmae)

----- NMAE:  3.485148514851485


## Cài đặt lại bài tập thực hành số 3 (CB)

### Read dataset

In [44]:
# Load dữ liệu movies_metadata.csv
movies_metadata = pd.read_csv(PATH+'/movies_metadata.csv')
movies_metadata

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circum...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",NaN,8844,tt0113497,en,Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- in...",...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]",NaN,15602,tt0113228,en,Grumpier Old Men,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming t...",...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, ...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself... and never let you forget it.,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'backdrop_path': '/7qwE57OVZmMJChBpLEbJEmzUydk.jpg'}",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home...",...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's In For The Surprise Of His Life!,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}]",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a storyline about a cult plays in his head.,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'nam

In [45]:
# Xuất ra overview của 5 bộ phim đầu tiên
movies_metadata[['overview']].head(5)

,overview
0,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circum..."
1,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- in..."
2,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming t..."
3,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, ..."
4,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home..."


In [46]:
# Replace NaN with an empty string
movies_metadata['overview'] = movies_metadata['overview'].fillna('')

### Pre-processing data

In [47]:
# Viết hàm tiền xử lý dữ liệu trên cột overview
def processing_data(data):
  # Chuyển về từ thường
  data = data.lower()
  # Xóa dấu câu, ký tự đặc biệt
  data = re.sub('\W+',' ', data)
  # Xóa khoảng trắng đầu và cuối câu
  data = data.strip()
  # Xóa stopword
  data = ' '.join([word for word in data.split() if word not in stopwords.words("english")])
  # Tách từ
  # word_tokens = word_tokenize(data)
  # ...
  return data

In [48]:
# Viết hàm tiền xử lý dữ liệu trên cột overview
def processing_overview(col):
  for i in tqdm(range(movies_metadata.shape[0])):
    try:
      col[i] = processing_data(col[i])
    except:
      print(f'Value of row {i} is {col[i]}')
  return col

In [61]:
# Xử lý dữ liệu trên cột overview
processing_overview(movies_metadata['overview']) 

Output hidden; open in https://colab.research.google.com to view.

In [50]:
# Sử dụng TfidfVectorizer để chuyển cột overview của các bộ phim về tf-idf và lưu vào biến overview_matrix
vectorizer = TfidfVectorizer(stop_words='english')
overview_matrix = vectorizer.fit_transform(movies_metadata['overview'])

In [51]:
# In ra shape của overview_matrix nếu = (45466, 75827) là đúng.
print(overview_matrix.shape)

(45466, 75827)


In [52]:
# Lấy overview_matrix_new có kích thước bằng 1 nửa kích thước của overview_matrix
overview_matrix_new = overview_matrix[:int(overview_matrix.shape[0]/2),:int(overview_matrix.shape[1]/2)]

In [53]:
print(overview_matrix_new.shape)

(22733, 37913)


In [54]:
# Tính toán cosine giữa các bộ phim với nhau bằng linear_kernel
cosine_sim = linear_kernel(overview_matrix_new, overview_matrix_new)
cosine_sim

array([[0.58098714, 0.        , 0.        , ..., 0.        , 0.01757945,
        0.        ],
       [0.        , 0.61125382, 0.02176041, ..., 0.03636249, 0.        ,
        0.        ],
       [0.        , 0.02176041, 0.45575756, ..., 0.        , 0.00642748,
        0.00629375],
       ...,
       [0.        , 0.03636249, 0.        , ..., 0.6609658 , 0.        ,
        0.        ],
       [0.01757945, 0.        , 0.00642748, ..., 0.        , 0.39788609,
        0.00550112],
       [0.        , 0.        , 0.00629375, ..., 0.        , 0.00550112,
        0.52920617]])

In [55]:
cosine_sim.shape

(22733, 22733)

In [56]:
# Đánh index cho các bộ phim bằng pd.Series() và lưu trong biến mapping
mapping = pd.Series(movies_metadata.index, index=movies_metadata['title']).drop_duplicates()
mapping

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
Subdue                         45461
Century of Birthing            45462
Betrayal                       45463
Satan Triumphant               45464
Queerama                       45465
Length: 45466, dtype: int64

In [57]:
# Viết hàm nhận vào index của bộ phim trả về tên của bộ phim
def get_title(index):
  Title = movies_metadata.iloc[index]['title']
  return Title

In [58]:
get_title(4)

'Father of the Bride Part II'

In [59]:
# Viết hàm trả và top 10 bộ phim sẽ đem đi khuyến nghị cho người dùng U. Người dung U đã xem bộ phim có tên là "Father of the Bride Part II"
def get_recommendations(title, cosine_sim=cosine_sim):
  # Get the index of the movie that matches the title
  idx = mapping[title]
  # Get the pairwsie similarity scores of all movies with that movie
  sim_scores = list(enumerate(cosine_sim[idx]))
  # Sort the movies based on the similarity scores
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  # Get the scores of the 10 most similar movies
  sim_scores = sim_scores[1:11]
  # Get the movie mapping
  movie_mapping = [i[0] for i in sim_scores]
  # Return the top 10 most similar movies
  return movies_metadata['title'].iloc[movie_mapping]

In [60]:
get_recommendations('Father of the Bride Part II')

6306           North to Alaska
13611      The Magic of Méliès
19801                  Babbitt
6571                     Kuffs
5005                   Wendigo
7097        The Out of Towners
6793       Father of the Bride
926      It's a Wonderful Life
13928               Visioneers
5571            All Night Long
Name: title, dtype: object